# Importing Spark

In [ ]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark").getOrCreate()
sc = spark.sparkContext

# Loading the data

In [ ]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.data')

# Exercice 1 - constructing the recommender system

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
df_ratings.printSchema()

In [ ]:
# split

In [ ]:
# fit ALS

In [ ]:
# evaluate

In [ ]:
# Generate top 10 movie recommendations for each user
# Generate top 10 user recommendations for each movie